In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.getcwd()

In [ ]:
train = pd.read_csv('data/training.1600000.processed.noemoticon.csv',
                       encoding = "ISO-8859-1", engine='python',header=None,
                      names=['sentiment','tweet_id','date','flag','user','text'])

### Exploratory Data Analysis

Find the shape of dataset:

In [ ]:
train.shape

**Looking at my target variable `sentiments`:**

In [ ]:
train['sentiment'].value_counts()

It appears that negative tweet is given a label of 0 and positive tweet is given a label of 4. For simplicity, I will convert the binary classification to [0,1].

In [ ]:
def convert_sent(x):
    if x == 4:
        return 1
    else:
        return -1

In [ ]:
train['sentiment'] = train['sentiment'].apply(convert_sent)
train['sentiment']

In [ ]:
# After conversion
train['sentiment'].value_counts()

**Looking at the tweet_id:** 

Hypothetically, the tweet id should be unique identifier of each tweet, so if we count the number of unique tweets, we should get back a value of 1600000.

In [ ]:
len(train['tweet_id'].unique())

But appearly that is not the case... Perhaps there are duplicated rows in the dataset:

In [ ]:
id_count = train['tweet_id'].value_counts()

In [ ]:
# Looking at one of these duplicated id 1753678185  
pd.options.display.max_colwidth = 90
train[train['tweet_id'] == 1753678185]

In [ ]:
train[train['tweet_id'] == 1984377787]

It appears they are indeed duplicate rows, with different lables of sentiments! I have decided to remove these rows from the datasets.

In [ ]:
# Find row indexes where the tweed id has exactly one entry
id_count[id_count.values == 1].index
train = train[train['tweet_id'].isin(id_count[id_count.values == 1].index)].reset_index()
train.head()

In [ ]:
# New shape
train.shape

**Looking at the `date` column:**

In [ ]:
train['date'].dtypes

In [ ]:
# Time Zone
area_code = train['date'].str[20:24]
area_code.value_counts()

It appears all of the time zones are recorded in Pacific Time.

In [ ]:
# Year
year = train['date'].str[24:29]
year.value_counts()

All of the tweets are from 2009.

**Looking at the `flag` column:**

In [ ]:
train['flag'].value_counts()

It appears all of the rows contain the value of `NO_QUERY`, hence this column would not bring any information to our model, hence I will be removing it.

In [ ]:
train = train.drop(['flag'],axis=1)

In [ ]:
train.shape

**Looking at the `user` column:**

In [ ]:
train['user'].value_counts()

We have 659,775 unique users in this dataset from 2009.

**Looking at the `text` column:**

In [ ]:
train['text'].value_counts().head()

In [ ]:
# Find text columns with hashtags
train['text'].str.contains('#').value_counts()

In [ ]:
# Find text columns with mentions
train['text'].str.contains('@').value_counts()

In [ ]:
# Find text columns with urls
train['text'].str.contains('http://').value_counts()